In [1]:
import pandas as pd
import json
import sys
import re
sys.path.append('..')

In [2]:
from llm_class import LLM

llm = LLM(model_id="bielik", key_path='../key.txt')

In [3]:
df = pd.read_csv("../data/dialog_witcher_dataset.csv", delimiter="|")
with open('../data/speakers_characteristics.json', 'r') as f:
    speaker_data = json.load(f) 

In [4]:
def extract_translation(llm_resposne):
    match = re.search(r'\[(.*?)\]', llm_resposne)
    if match:
        return match.group(1)
    return None

In [5]:
prompt_single_line = """
Oto kontekst: Wypowiedź mówiona jest przez {speaker} ({gender}).

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: {text_to_translate}
"""

In [6]:
prompt_multiple_lines = """ 
Oto kontekst: poniżej znajduje się kawałek {monolog_or_dialog}{speakers}

{dialog}

Na podstawie podanego {monolog_or_dialog} oraz kontekstu, przetłumacz tylko i wyłącznie {first_or_second} wypowiedź na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Na podstawie podanego {monolog_or_dialog} oraz kontekstu, przetłumacz tylko i wyłącznie {first_or_second} wypowiedź na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: {text_to_translate}
"""

In [7]:
prompt_multiple_linesv2 = """ 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: Dubhenn haern am glâdeal.
Tłumaczenie: [Dubhenn haern am glâdeal.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Oto kontekst: poniżej znajduje się kawałek {monolog_or_dialog}{speakers}

{dialog}

Na podstawie podanego {monolog_or_dialog} oraz kontekstu, przetłumacz tylko i wyłącznie {first_or_second} wypowiedź na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych: {text_to_translate}

"""

In [8]:
prompt_single_linev2 = """
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Oto kontekst: Wypowiedź mówiona jest przez {speaker} ({gender}).

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: {text_to_translate}
"""

In [10]:
prompt_default = """
Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: {en_text}
"""

In [9]:
def choose_and_format_prompt(speaker1, speaker2, speaker3, prev_text, text_to_translate, next_text):

    if speaker2 == None:
        print("no main speaker")
        return None
    
    speaker2_gender = speaker_data.get(speaker2, {}).get('gender')

    #jezeli pojedyncza wypowiedz
    if speaker1 == None and speaker3 == None:
        return prompt_single_linev2.format(speaker=speaker2, gender=speaker2_gender, text_to_translate=text_to_translate)
    
    #jezeli weicej niz jedna wypowiedz
    speaker1_gender = speaker_data.get(speaker1, {}).get('gender')
    speaker1_gender = f" ({speaker1_gender})" if speaker1_gender and speaker1_gender.lower() != 'unknown' else ""

    speaker2_gender = f" ({speaker2_gender})" if speaker2_gender and speaker2_gender.lower() != 'unknown' else ""

    speaker3_gender = speaker_data.get(speaker3, {}).get('gender')
    speaker3_gender = f" ({speaker3_gender})" if speaker3_gender and speaker3_gender.lower() != 'unknown' else ""

    monolog_or_dialog = ""
    speakers = ""
    dialog = ""
    first_or_second = ""

    speakers = [speaker1, speaker2, speaker3]
    non_none_speakers = [s for s in speakers if s is not None]
    unique_speakers = set(non_none_speakers)
    is_monolog = len(unique_speakers) == 1

    #tworzenie dialogu
    if speaker1:
        dialog += f"{speaker1} - {prev_text}\n"
        #piersza czy druga wypowiedz
        first_or_second = "drugą"
    else:
        first_or_second = "pierwszą"

    if speaker2:
        dialog += f"{speaker2} - {text_to_translate}\n"

    if speaker3:
        dialog += f"{speaker3} - {next_text}\n"

    #handling monolog
    if is_monolog:
        monolog_or_dialog = "monologu"
        speakers = f" wypowiadany przez {speaker2}{speaker2_gender}"
    #handling dialog
    else:
        monolog_or_dialog = "dialogu"
        if (speaker1 != speaker2 and speaker3 == None) or (speaker1 != speaker2 and speaker3 == speaker2) or (speaker1 != speaker2 and speaker1==speaker3):
            speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender}"
        elif (speaker3 != speaker2 and speaker1 == None) or (speaker3 != speaker2 and speaker1 == speaker2) or (speaker3 != speaker2 and speaker3==speaker1):
            speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker3}{speaker3_gender}"
        else:
            speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender} oraz z {speaker3}{speaker3_gender}"

    return prompt_multiple_linesv2.format(monolog_or_dialog=monolog_or_dialog, speakers=speakers, dialog=dialog, first_or_second=first_or_second, text_to_translate=text_to_translate)

In [12]:
df = df.where(pd.notna(df), None)

In [13]:
sampled_df = df.sample(n=10, random_state=532423).reset_index(drop=True)
sampled_df

,speaker,prev_text,prev_speaker,next_text,next_speaker,en_text,pl_text
0,Skellige Blacksmith Helper 01,Sorry to hear it.,Geralt,Eh? What is it?,Skellige Blacksmith Helper 01,"Sea swallowed his body… I'll not bury him, he'...",Morze nie wyrzuciło ciała... Nie pochowam go. ...
1,Baron Bandit Weak 03,Bollocks.,Baron Bandit Weak 03,"Love to, but not till the morn.",Baron Bandit Weak 03,Whaddaya say we hit the high road? Set out in ...,A jakby tak na gościniec podlecieć? Zarobku po...
2,Geralt,I made a mistake. I know this now. Swear by Fr...,Skellige Villager 02,None,None,"Absolutely, never. Soon as we're done talking,...","Tak, nigdy więcej. Bo jak tylko skończymy rozm..."
3,Geralt,And yet you came in.,Cirilla,Speaking of which - they're probably getting a...,Geralt,Who listens to dwarves these days?,Kto by tam słuchał krasnoludów.
4,Baron,"Sometimes miscarried fetuses, if they don't ge...",Geralt,A cursed creature that draws strength from kil...,Geralt,"Into, fucking, what...?","O czym ty, kurwa... W co?"
5,Temerian Guard 02,Piss off?,Temerian Guard 02,Hit the freak!,Temerian Guard 02,You a thrill-seeker?,Chojraczysz?
6,Geralt,What else could he want from two witchers? Go ...,Vesemir,Need help?,Vesemir,Griffin's abandoned its lair. Gotta make a lur...,Gryf porzucił leże. Trzeba zrobić wabik.
7,Skellige Villager 02,None,None,Tis a crime. The gods will punish us!,Skellige Villager 05,Killed 'em in cold blood…,Zabili ich z zimną krwią...
8,Geralt,A shame events like this are so rare. Without ...,Triss,I'll be sure to admire your valiant suffering ...,Triss,Argh. Damn doublet's chafing my armpits. And i...,Zaraza... Ten dublet ciśnie pod pachami... I c...
9,Geralt,See? Ravvy's on board. Whaddaya say we add som...,Novigrad Thug Strong 02,Tsk. This witcher's no fun.,Novigrad Thug Strong 02,I got no use for Ravvy.,Ravik mnie nie interesuje.


In [10]:
for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']

    original_translation = row['pl_text']

    prompt = choose_and_format_prompt(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)

    print("Prompt:")
    print(prompt)
    print()
    print()

    response = llm.prompt_chat_custom_temperature(prompt=prompt, temperature=0.0)

    print("Response:")
    print(response)
    print()
    print()

    translation = extract_translation(response)

    if translation == None:
        print(":(((")
        continue

    print("Translation: ", translation)
    print("Original: ", original_translation)
    print()
    print()
    print("============================================")   
    print()
    print()
    

Prompt:
 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: Dubhenn haern am glâdeal.
Tłumaczenie: [Dubhenn haern am glâdeal.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Oto kontekst: poniżej znajduje się kawałek dialogu, rozmawia w nim Skellige Blacksmith Helper 01 (mężczyzna) z Geralt (mężczyzna)

Geralt - Sorry to hear it.
Skellige Blacksmith Helper 01 - Sea swallowed his body… I'll not bury him, he'll not feast with his ancestors.
Skellige Blacksmith Helper 01 - Eh? What is it?


Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko 

In [10]:
sampled_df = df.sample(n=2000, random_state=444).reset_index(drop=True)
sampled_df

,speaker,prev_text,prev_speaker,next_text,next_speaker,en_text,pl_text
0,Cirilla,When I learned Geralt had found you--,Dandelion,Where've you two been?,Zoltan Chivay,"Glad you to see you in one piece, too, Dandelion.","Cieszę się, że też jesteś cały."
1,Geralt,"The sharp one. C'mon, hurry up.",Eskel,Mean you don't know?,Eskel,Yen tell you why she wants this?,"Yen wspomniała, po co jej to?"
2,Geralt,NaN,NaN,Yeshhh?,Novigrad Rich Man 03,Greetings.,Witaj.
3,Witch Hunter 01,"Gladly. Gotta warn you, though - you're not go...",Geralt,"Another time, maybe.",Geralt,Rally to me!,Do mnie!
4,Ermion,How's Cerys handling things?,Geralt,"Everyone, Lugos excepted, declared their suppo...",Ermion,"Has the makings of an excellent queen, if she ...","Byłaby świetną królową, gdyby udało jej się wy..."
...,...,...,...,...,...,...,...
1995,Geralt,You are free!,Yennefer,Yes. It's all over.,Yennefer,Storm's over.,Burza ucichła.
1996,Redanian Soldier 01,On whose orders?,Geralt,Mhm. I've got a pass.,Geralt,"By order of His Grace, King Radovid himself. W...",Samego miłościwego króla Radowida. A to oznacz...
1997,Geralt,Dare harm me... and against you will rise... a...,Ghost In The Tree,This is my prison... A fortress besieged...,Ghost In The Tree,I was attacked.,Zostałem zaatakowany.
1998,Sorceress 03,I'll not leave here. I'm no fool!,Mage 01,Have you seen what's happening out there?! The...,Mage 01,Berthold…,Bertold...


In [11]:
for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']

    original_translation = row['pl_text']

    prompt = choose_and_format_prompt(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)

    response = llm.prompt_chat_custom_temperature(prompt=prompt, temperature=0.0)

    translation = extract_translation(response)

    if translation == None:
        sampled_df.at[index, f'dialog_prototype'] = ""
        continue

    sampled_df.at[index, f'dialog_prototype'] = translation

    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/dialog_prototype_experimentsv2.csv", sep="|")

Processed 100 out of 2000 samples...
Processed 200 out of 2000 samples...
Processed 300 out of 2000 samples...
Processed 400 out of 2000 samples...
Processed 500 out of 2000 samples...
Processed 600 out of 2000 samples...
Processed 700 out of 2000 samples...
Processed 800 out of 2000 samples...
Processed 900 out of 2000 samples...
Processed 1000 out of 2000 samples...
Processed 1100 out of 2000 samples...
Processed 1200 out of 2000 samples...
Processed 1300 out of 2000 samples...
Processed 1400 out of 2000 samples...
Processed 1500 out of 2000 samples...
Processed 1600 out of 2000 samples...
Processed 1700 out of 2000 samples...
Processed 1800 out of 2000 samples...
Processed 1900 out of 2000 samples...
Processed 2000 out of 2000 samples...
